In [1]:
import pandas as pd
df1=pd.read_csv('../data/01_output_200_300.csv')
df2=pd.read_csv('../data/01_output_300_400.csv')
df3=pd.read_csv('../data/01_output_400_500.csv')
df4=pd.read_csv('../data/01_output_500_600.csv')

# merge the previous dfs into one df
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   languageCode        400 non-null    object
 1   INTERNAL_ID         400 non-null    object
 2   CUSTOMER_ID         400 non-null    object
 3   PARSED_CUST_ID      400 non-null    int64 
 4   TASK_ID             400 non-null    object
 5   BATCH_ID            400 non-null    object
 6   prompt              400 non-null    object
 7   schema              400 non-null    object
 8   response            400 non-null    object
 9   MR_EVAL_SUB_SCHEMA  400 non-null    object
 10  REFERENCED_JSON     400 non-null    object
dtypes: int64(1), object(10)
memory usage: 34.5+ KB


In [2]:
import pandas as pd

# Function to count the number of properties in True and the number of properties in False of JSON
# json example
#[
#    {
#        "act_summaries": {
#            "type": "array",
#            "referenced": true,
#            "text_reference": "The prompt includes an 'Act Summaries' section with multiple acts (Act 1, Act 2, etc.) describing their content.",
#            "items": {
#                "type": "object",
#                "properties": {
#                    "act_number": {
#                        "type": "integer",
#                        "referenced": true,
#                        "text_reference": "Each act is identified by its number in phrases like 'Act 1:', 'Act 2:', etc., which implies a numerical identifier."
#                    },
#                    "summary": {
#                        "type": "string",
#                        "referenced": true,
#                        "text_reference": "For every act, a narrative description is provided (e.g., 'The play opens with guards encountering the ghost of King Hamlet' for Act 1) summarizing the act."
#                    },
#                    "scene_summaries": {
#                        "type": "array",
#                        "referenced": true,
#                        "text_reference": "The prompt includes detailed 'Scene Summaries' for selected acts (e.g., Act 1, Scene 1; Act 1, Scene 5; Act 3, Scene 2; Act 5, Scene 1; Act 5, Scene 2).",
#                        "items": {
#                            "type": "object",
#                            "properties": {
#                                "scene_number": {
#                                    "type": "integer",
#                                    "referenced": true,
#                                    "text_reference": "Each scene is identified by a number in the text such as 'Scene 1' or 'Scene 2' attached to its act (e.g., 'Act 1, Scene 1')."
#                                },
#                                "setting": {
#                                    "type": "string",
#                                    "referenced": true,
#                                    "text_reference": "Settings for scenes are explicitly described (e.g., 'Elsinore', 'The battlements', 'A hall in the castle', 'A churchyard')."
#                                },
#                            }
#                        }
#                    }
#                },
#                "required": [
#                    "act_number",
#                    "summary"
#                ]
#            }
#        },
#        "property_name": "act_summaries"
#    },
#
#]
def count_referenced_properties(json_data):
    true_count = 0
    false_count = 0
    referenced_true = []
    referenced_false = []
    
    def count_references(obj, parent_key=''):
        nonlocal true_count, false_count
        if isinstance(obj, dict):
            for key, value in obj.items():
                if key == 'referenced':
                    if value:
                        true_count += 1
                        referenced_true.append(parent_key)
                    else:
                        false_count += 1
                        referenced_false.append(parent_key)
                else:
                    count_references(value, key if parent_key == '' else f"{parent_key}.{key}")
        elif isinstance(obj, list):
            for item in obj:
                count_references(item, parent_key)
    
    count_references(json_data)
    return true_count, false_count, remove_properties_suffix(referenced_true), remove_properties_suffix(referenced_false)

# create a function to count the number of properties of the json schema
def count_properties_schema(json_schema):
    property_names = []
    
    def count_props(obj, parent_key=''):
        if isinstance(obj, dict):
            for key, value in obj.items():
                if key == 'properties':
                    property_names.extend([f"{parent_key}.{k}" if parent_key else k for k in value.keys()])
                count_props(value, key if parent_key == '' else f"{parent_key}.{key}")
        elif isinstance(obj, list):
            for item in obj:
                count_props(item, parent_key)
    
    count_props(json_schema)
    return remove_properties_suffix(property_names)

def remove_properties_suffix(strings):
    return [s.replace('properties.', '') for s in strings]

In [3]:
# read the first value of referenced_json
import json
import ast

# read the first value of schema
schema = df['schema'].iloc[0]
json_schema = json.loads(schema)
#print(json.dumps(json_schema, indent=4))

referenced_json = df['REFERENCED_JSON'].iloc[0]

def extract_referenced_json(referenced_json):
    referenced_json = ast.literal_eval(referenced_json)
    referenced_json = json.loads(json.dumps(referenced_json))
    return json.dumps(referenced_json, indent=4)
print(extract_referenced_json(referenced_json))





[]


In [4]:
def summary(json_schema,referenced_json):
# Example usage with the provided JSON schema
    property_count = count_properties_schema(json_schema)

    #print(f"List Property Schema: {property_count}")

    # Example usage with the provided JSON data
    true_count, false_count, referenced_true, referenced_false = count_referenced_properties(referenced_json)
    #print(f"True count: {true_count}, False count: {false_count}")
    #print("Referenced properties True:", referenced_true)
    #print("Referenced properties False:", referenced_false)

    # Find the difference between property_count and referenced_true
    difference = list(set(property_count) - set(referenced_true))
    match=len(property_count)-len(difference)

    #create a list of intersect of referenced_false and property_count
    referenced_false = list(set(referenced_false) & set(property_count))

    difference_missing = list(set(difference) - set(referenced_false))
    return {"schema_count": len(property_count),
            "referenced_true": match,
            "score_reference": match/len(property_count),
            "list_missing_properties_prompt": difference_missing,
            "missing_properties_prompt": len(difference_missing),
            "list_referenced_false": referenced_false,
            "referenced_false": len(referenced_false)
        }

print(summary(json_schema,referenced_json))

#function to format the json in this output string 

#Number of Properties in the Schema:
#Properties mentioned in the prompt (according to the LLM):
#    •   Count:
#Properties not mentioned in the prompt (according to the LLM):
#    •   Count:
#    •   List:
#Properties not evaluated by the model:
#    •   Count:
#    •   List: (edited)
def format_summary_output(summary_dict):
    output = (
        f"Number of Properties in the Schema: {summary_dict['schema_count']}\n"
        f"Properties mentioned in the prompt (according to the LLM):\n"
        f"    •   Count: {summary_dict['referenced_true']}\n"
        f"Properties not mentioned in the prompt (according to the LLM):\n"
        f"    •   Count: {summary_dict['referenced_false']}\n"
        f"    •   List: {', '.join(summary_dict['list_referenced_false'])}\n"
        f"Properties not evaluated by the model:\n"
        f"    •   Count: {summary_dict['missing_properties_prompt']}\n"
        f"    •   List: {', '.join(summary_dict['list_missing_properties_prompt'])}\n"        
    )
    return output

formatted_summary = format_summary_output(summary(json_schema, referenced_json))
print(formatted_summary)


{'schema_count': 39, 'referenced_true': 0, 'score_reference': 0.0, 'list_missing_properties_prompt': ['items.timeframe.ongoing', 'items.involved_organizations.items.role', 'items.impact_and_outcomes.quantitative_metrics.items.baseline_value', 'items.impact_and_outcomes.quantitative_metrics.items.reporting_period', 'items.impact_and_outcomes.qualitative_description', 'items.funding_sources.items.year', 'items.involved_organizations.items.organization_name', 'items.impact_and_outcomes.quantitative_metrics.items.target_value', 'items.keywords', 'items.related_resources', 'items.funding_sources.items.funding_type', 'items.contact_information.contact_person', 'items.status', 'items.funding_sources.items.amount', 'items.funding_sources.items.currency', 'items.contact_information.phone', 'items.contact_information', 'items.target_beneficiaries', 'items.geographic_scope', 'items.related_resources.items.description', 'items.funding_sources.items.source_name', 'items.impact_and_outcomes.quantita

In [5]:
df['REFERENCED_JSON_FORMATED2']=df['REFERENCED_JSON'].apply(extract_referenced_json)
# create a new column summary_json by using the function summary with the column REFERENCED_JSON_FORMATED2 and schema
df['summary_json'] = df.apply(lambda row: summary(json.loads(row['schema']), json.loads(row['REFERENCED_JSON_FORMATED2'])), axis=1)
df['summary'] =df['summary_json'].apply(format_summary_output)


df['score_reference'] = df['summary_json'].apply(lambda x: x['score_reference'])
df['missing_properties'] = df['summary_json'].apply(lambda x: x['missing_properties_prompt'])
df['schema_properties'] = df['summary_json'].apply(lambda x: x['schema_count'])
df['false_properties'] = df['summary_json'].apply(lambda x: x['referenced_false'])
print(df[['score_reference','missing_properties','schema', 'REFERENCED_JSON', 'summary']])


     score_reference  missing_properties  \
0           0.000000                  39   
1           0.635135                  22   
2           1.000000                   0   
3           0.696970                   0   
4           0.343750                  42   
..               ...                 ...   
395         0.191617                  93   
396         0.000000                  26   
397         0.384615                  16   
398         0.000000                  18   
399         0.761194                  15   

                                                schema  \
0    {"type": "array", "items": {"type": "object", ...   
1    {"type": "object", "properties": {"workflow_na...   
2    {"type": "object", "properties": {"remoteWorkT...   
3    {"type": "object", "properties": {"substitutio...   
4    {"type": "object", "properties": {"course_name...   
..                                                 ...   
395  {"type": "object", "properties": {"frameworkNa...   
396  {"

In [6]:
df2=df.rename(columns={'REFERENCED_JSON_FORMATED2': 'reference_JSON',
                   'INTERNAL_ID':'internal_id',
                   'TASK_ID': 'TASK'})

# keep the columns needed
df2 = df2[['TASK','languageCode', 'internal_id', 'prompt', 'schema', 'reference_JSON', 'summary','score_reference','schema_properties','missing_properties','false_properties']]


# remove rows that contains "prompt_related_score": 0," in summary column
#df = df[~df['summary'].str.contains('"prompt_related_score": 0,')]

# replace all the value in 'languageCode' for "en_US"
df2['languageCode'] = 'en_US'

# save to csv
df2.to_csv('../data/01_output_200_600_summary.csv',index=False)